<a href="https://colab.research.google.com/github/ranieri-unimi/faces-malchiodi-2022/blob/main/faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [1]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 68.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c148bfc1698a7e7546f68363fef059ec38c0a62972b720fbaabe94b9d34268ca
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [3]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

100% 9.75G/9.76G [00:57<00:00, 191MB/s]
100% 9.76G/9.76G [00:57<00:00, 184MB/s]


## ukraine

In [4]:
SAMPLE_SIZE = 2048

In [5]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [6]:
import pyspark
import findspark

In [7]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## data cleaning

In [8]:
df = df[df.language == 'en']

In [9]:
df = df.text.tolist()

In [10]:
df = random.sample(df, SAMPLE_SIZE) if SAMPLE_SIZE else df

In [11]:
def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’”“', ' '*(len(string.punctuation)+4)))  # puntctuation to spaces
    tweet = ' '.join(tweet.strip().split())
    return tweet

## hadoop instance

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [13]:
rdd = sc.parallelize(enumerate(df))

In [14]:
rdd.take(7)

[(0,
  "Twitter  trends are a mess right now. We've got millions of people spreading misinformation and fake news. There are no reliable sources on #RussiaUkraineConflict: mostly pro-#Ukraine blatant propaganda, and a lot of #Antiputinism. Smartphones are dangerous at times like these."),
 (1,
  '#Moldova President calls out flagrant violation of intl. law by #Russia, asks gov to declare state of emergency this evening, police patrols to intensify, requests people to be understanding and help Ukrainians. “We are ready to accommodate several tens of thousands of people”. https://t.co/BVmU1hbZ4r'),
 (2,
  "Until #Kharkiv surrenders #Russians soldiers don't want to fight against #Ukraine, these are fraternal Slavic races, reasons to fight too are absurd. https://t.co/eeCrcrywsM"),
 (3,
  'Find it so true man!\n#UkraineRussiaWar #UNSC #RussiaUkraineWar #Zelensky https://t.co/bszlLrHMyf'),
 (4,
  'Alleged video of #Russian helicopters attacking Hostomel town in the #Kyiv Oblast\n\n#RussiaUk

In [15]:
def shiftkv(x) : return (x[-1], x[0])

In [16]:
dataPipe = (rdd
            .mapValues(preprocess_tweet_text)
            .map(shiftkv)
            .reduceByKey(min)
            .map(shiftkv)
)

In [17]:
dataPipe.take(7)

[(1,
  'moldova president calls out flagrant violation of intl law by russia asks gov to declare state of emergency this evening police patrols to intensify requests people to be understanding and help ukrainians we are ready to accommodate several tens of thousands of people'),
 (2,
  'until kharkiv surrenders russians soldiers don t want to fight against ukraine these are fraternal slavic races reasons to fight too are absurd'),
 (3, 'find it so true man ukrainerussiawar unsc russiaukrainewar zelensky'),
 (4,
  'alleged video of russian helicopters attacking hostomel town in the kyiv oblast russiaukraineconflict russia ukraine'),
 (8,
  'azerbaijan is have started photographing the military identity cards of armenian soldiers killed in artsakh in 2020 amp distributing them on 🇺🇦n websites as proof of the participation of the armenian military in hostilities in ukraine don t publish fake news re public press'),
 (9,
  'zelenskyyua s tv address to the russian people might be the most m

### foos

In [18]:
!pip install crc16
import crc16
import binascii

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for crc16: filename=crc16-0.1.1-cp37-cp37m-linux_x86_64.whl size=22881 sha256=14e04903846816405ea17c20bae996c4d06e3f4373a2ba8027ebd2f4ce5216bf
  Stored in directory: /root/.cache/pip/wheels/2b/28/c3/e6303cf55de8a79c8c4b1528bd4b5f577d85debdbe916c0e2c
Successfully built crc16


### shingles

In [19]:
SH_LEN = 9

In [20]:
shigles = dataPipe.flatMap(lambda x : [(x[0], x[-1][i:i+SH_LEN]) for i in range(len(x[-1])-SH_LEN)]).distinct()

In [21]:
shigles.take(7)

[(1, 'moldova p'),
 (1, 'oldova pr'),
 (1, 'dova pres'),
 (1, 'ova presi'),
 (1, 'va presid'),
 (1, 'a preside'),
 (1, ' presiden')]

In [22]:
def hashval(x):
  bval = bytes(x, 'utf-8')
  hval = binascii.crc32(bval) # 4 bytes
  return hval

In [23]:
charactMx = shigles.mapValues(hashval).distinct()

In [24]:
charactMx.take(7)

[(1, 951947353),
 (1, 2387657533),
 (1, 1661334129),
 (1, 2597531057),
 (1, 3999582881),
 (1, 605588507),
 (1, 1923190723)]

### MinHashing

In [25]:
def RN(stop, start = 1) : return random.randrange(start,stop)
def RB(B = 4) : return RN(2**(8*B-1),0)*2+1
def PF(x, scale, shift, base) : return (scale * x + shift) % base

In [26]:
import random
n_permfoo, b_bands = 15, 3

(1 / b_bands)**(1 / (n_permfoo//b_bands)) # threshold

0.8027415617602307

In [27]:
params = [ {'shift':RB(), 'scale':RB(), 'base':2**(8*4)} for _ in range(n_permfoo) ]
params

[{'base': 4294967296, 'scale': 3781094775, 'shift': 277300205},
 {'base': 4294967296, 'scale': 1203808171, 'shift': 2718487213},
 {'base': 4294967296, 'scale': 3623957577, 'shift': 3464738027},
 {'base': 4294967296, 'scale': 183732079, 'shift': 2627346995},
 {'base': 4294967296, 'scale': 697574343, 'shift': 800712657},
 {'base': 4294967296, 'scale': 3302116531, 'shift': 668252617},
 {'base': 4294967296, 'scale': 1341339701, 'shift': 698456999},
 {'base': 4294967296, 'scale': 2625907173, 'shift': 4133601277},
 {'base': 4294967296, 'scale': 85380745, 'shift': 4191611121},
 {'base': 4294967296, 'scale': 2757249719, 'shift': 1892644733},
 {'base': 4294967296, 'scale': 1369746573, 'shift': 1494008949},
 {'base': 4294967296, 'scale': 347452949, 'shift': 1218373361},
 {'base': 4294967296, 'scale': 3737832663, 'shift': 1183785389},
 {'base': 4294967296, 'scale': 1489357413, 'shift': 2891422953},
 {'base': 4294967296, 'scale': 2321922795, 'shift': 2473456359}]

In [28]:
def gen_perm(x):
  doc, shingle = x
  return [ ((doc, h), (shingle, h) ) for h in range(n_permfoo)]

In [29]:
minHashSignMx = (charactMx
                 .flatMap(gen_perm)
                 .reduceByKey(lambda a, b : a if PF(a[0], **params[a[-1]]) < PF(b[0], **params[a[-1]]) else b)
                 .mapValues(lambda v : v[0])
)
# (doc, hash) , shingle

In [30]:
minHashSignMx.take(7)

[((1, 1), 1886652996),
 ((1, 3), 1828456720),
 ((1, 5), 3703021807),
 ((1, 7), 1072751133),
 ((1, 9), 2918359914),
 ((1, 11), 652022045),
 ((1, 13), 322172028)]

### LSH

In [31]:
scale, shift = RN(b_bands), RN(b_bands)
bandmap = (lambda x : PF(x, scale, shift, b_bands))

In [32]:
def band_expand(x):
  (doc, h), v = x
  return ((doc, bandmap(h)), (h, v))

def band_reduct(x):
  (doc, band), a = x
  return ((hash(tuple(a)), band), doc)

In [33]:
bandMx = (minHashSignMx              # (doc, hashperm) , valshin
          .map(band_expand)             # (doc, band) , (hashperm , valshin)
          .groupByKey().mapValues(list)    # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...]
          .mapValues(lambda a : sorted(a, key=lambda x: x[0]))   # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...] but sorted
          .mapValues(lambda a : [v for _, v in a] )      # (doc, band) , [valshin, valshin, valshin]
)

In [34]:
bandMx.take(7)

[((1, 1), [1886652996, 3780031641, 1072751133, 3525282744, 322172028]),
 ((2, 2), [1333213944, 874229319, 2456131725, 4090047460, 3435266648]),
 ((2, 0), [1548488324, 2690361315, 315579151, 2506062748, 4165066784]),
 ((3, 1), [2836851489, 1577500677, 4203223442, 137454761, 2837494594]),
 ((4, 2), [134623952, 966194064, 471681518, 3938065016, 519034470]),
 ((4, 0), [2653825398, 1223856657, 966194064, 1806892112, 267726213]),
 ((8, 2), [1889003806, 990726201, 4271650572, 2244851893, 3435266648])]

In [35]:
bandBuckets = (bandMx
    .map(band_reduct)                           # (HHH_valshin, band), doc
    .groupByKey().mapValues(list)           # (HHH_valshin, band), [doc, doc, doc]
)

In [36]:
lst = bandBuckets.filter(lambda x : len(x[1])>1).collect()

In [37]:
lst

[((-5082129318915723509, 1), [31, 630]),
 ((-949560924248017008, 0), [36, 390]),
 ((6454236028692777106, 0), [90, 1050]),
 ((3068132518927323487, 2), [196, 1467]),
 ((913465999955253914, 0), [834, 355]),
 ((1514760888328223531, 1), [1033, 1045]),
 ((7803105975657921354, 1), [1199, 1734]),
 ((-6208552943985150898, 0), [1734, 1199]),
 ((6706672621367873397, 1), [201, 1404]),
 ((3550413957492532448, 1), [503, 1447]),
 ((9160511746229587954, 1), [637, 1469]),
 ((8055471250737602757, 2), [1404, 201]),
 ((-1883484680940027095, 1), [36, 390]),
 ((8121972216558314202, 1), [90, 1050]),
 ((8416296035383183707, 0), [591, 67]),
 ((-2694771494769646978, 1), [712, 232]),
 ((-2540336970540313367, 0), [1033, 1045]),
 ((-1457824966954666694, 2), [2011, 1197]),
 ((-8739385336650314657, 0), [503, 1447]),
 ((7886086883478047490, 2), [12, 599]),
 ((197303549563865873, 0), [12, 599]),
 ((4654249619368547339, 2), [36, 390]),
 ((8452797353214569670, 2), [220, 586]),
 ((6682962306036711474, 1), [241, 1389]),
 

In [38]:
for v in dict(lst).values():
  for i in v:
    print(df[i])
    print('---------------')
  print('___________________________________________________________________________________________')

@coingecko @NEARProtocol We want to live in peace. We don't want war. This is a NFT for brotherhood.
https://t.co/8n7xH4llEW
@opensea @opensea__nft #openseanft #opensea #NFT #NFTs #Ukraina #Russia #RussiaUkraineConflict #RussiaUkraine #UkraineRussia #war #NFTCommunity #nftcollector #NoWar #nowarinukraine https://t.co/gzGkCfHonx
---------------
@babelonNFT We want to live in peace. We don't want war. This is a NFT for brotherhood.
https://t.co/8n7xH4llEW
@opensea @opensea__nft #openseanft #opensea #NFT #NFTs #Ukraina #Russia #RussiaUkraineConflict #RussiaUkraine #UkraineRussia #war #NFTCommunity #nftcollector #NoWar #nowarinukraine https://t.co/qL6Y38YAIM
---------------
___________________________________________________________________________________________
RT, SPREAD AND SHARE, YOU CAN HELP UKRAINE #Ukraine #Russia https://t.co/4Lq1Jjs6wc
---------------
SPREAD AND SHARE, YOU CAN HELP UKRAINE #Ukraine #Russia https://t.co/rp2IFCKMi3
---------------
_________________________________